In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n!python3 -m pip install pandas\n!pip install db-dtypes\n")

- IMPORT THE LIBRARIES YOU WILL USE
------------------------------------------
You only need to import packages one time per notebook session. To keep your
notebook clean and organized you can handle all imports at the top of your file.
The following are included for example purposed, feel free to modify or delete 
anything in this section.

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import datetime,itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', "\n#- INSTALL ADDITIONAL LIBRARIES IF REQUIRED\n#------------------------------------------\n# This is normally not required. The hub environment comes preinstaled with \n# many packages that you can already use without setup. In case there is some\n# other library you would like to use that isn't on the list you run this command\n# once to install them.  If it is already installed this command has no effect.\n\n!python3 -m pip install pandas\n!pip install db-dtypes\n")

In [ ]:
get_ipython().system('pip install pmdarima --quiet')
import pmdarima as pm

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [ ]:
def example_function():
    print('Hello World')

In [ ]:
def print_column_info(df):
    print(f'No. of columns: {len(df.columns)}')
    for col in df.columns:
        print(len(df[col].unique()),col,df[col].dtypes)
    print()

In [ ]:
def print_na_info(df):
    for col in df.columns:
        print(df[col].isnull().sum(),col,df[col].dtypes)

In [ ]:
def get_datetime(week_no):
    date = datetime.datetime.strptime("2022-"+str(week_no)+"-1","%Y-%W-%w")
    #print(date)
    return pd.to_datetime(date,format="%Y-%m-%d")

In [ ]:
def sarimax_gridsearch(ts, pdq, pdqs, maxiter=100, freq='D',disp=False):
    # Run a grid search with pdq and seasonal pdq parameters and get the best BIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            #try:
            mod = SARIMAX(ts,order=comb,
                          seasonal_order=combs,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

            output = mod.fit(maxiter=maxiter,disp=False) 
            ans.append([comb, combs, output.bic])
            #print('SARIMAX {} x {}12 : BIC Calculated ={}'.format(comb, combs, output.bic))
            #except:
            #    continue
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'bic'])
    ans_df = ans_df.sort_values(by=['bic'],ascending=True)[0:5]
    
    return ans_df

In [ ]:
def check_stationary(data,Print=0):
    adft = adfuller(data,autolag="AIC")
    output_df = pd.DataFrame({"Values":[adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]  , "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                                            "critical value (1%)", "critical value (5%)", "critical value (10%)"]})

    critical_value = adft[4]['5%']
    if Print==1:
        print(output_df)
    if adft[1] < 0.05 and adft[0] < critical_value:
        return 1
    else:
        return 0

In [ ]:
def diff_inv(series, last_observation):

    series_undifferenced = series.copy()

    series_undifferenced.iat[0] = series_undifferenced.iat[0] + last_observation

    series_undifferenced = series_undifferenced.cumsum()

    return series_undifferenced

In [ ]:
def loss(pred,actual):
    pred = np.round(pred)
    errors = abs(actual-pred)
    print(f'Mean Absolute Error: {round(np.mean(errors), 2)}')
    print(f'Mean squared Error: {round(np.mean(errors**2), 2)}')
    mape = 100 * (errors/actual)
    # Calcualte and display accuracy
    accuracy = 100 - np.mean(mape)
    print(f'Accuracy: {round(accuracy, 2)}%.')

BIGQuery

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
#query = """
#SELECT 
#week_number,
#cases 
#FROM `ironhacks-data.ironhacks_training.covid19_cases`
#Where week_number between 1 and 3
#order by week_number
#"""
print("Datasets available:")
for dataset in list(bigquery_client.list_datasets()):
    print(dataset.dataset_id)
    if dataset.dataset_id == "ironhacks_competition":
        mydataset = dataset

In [ ]:
print("\nTables available:")
for table in bigquery_client.list_tables("ironhacks_competition"):
    print(table.table_id)

In [ ]:
query1 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""
query2 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
query3 = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.wage_data`
"""

In [ ]:
query_job1 = bigquery_client.query(query1)
query_job2 = bigquery_client.query(query2)
query_job3 = bigquery_client.query(query3)

In [ ]:
prediction_data = query_job1.to_dataframe()
unemployment_data = query_job2.to_dataframe()
wage_data = query_job3.to_dataframe()

In [ ]:
##Dumping the df to csv
week = "week2"
prediction_data.to_csv("prediction_data_"+week+".csv",index=False)
unemployment_data.to_csv("unemployment_data_"+week+".csv",index=False)
wage_data.to_csv("wage_data_"+week+".csv",index=False)

In [ ]:
merged_data = pd.merge(unemployment_data,wage_data[["uu_id","average_wage"]],on="uu_id",how="left")
merged_data = merged_data.drop_duplicates()

In [ ]:
query4 = """
SELECT table_id,
DATE(TIMESTAMP_MILLIS(creation_time)) AS creation_date,
DATE(TIMESTAMP_MILLIS(last_modified_time)) AS last_modified_date,
row_count,
size_bytes,
CASE
    WHEN type = 1 THEN 'table'
    WHEN type = 2 THEN 'view'
    WHEN type = 3 THEN 'external'
    ELSE '?'
END AS type,
TIMESTAMP_MILLIS(creation_time) AS creation_time,
TIMESTAMP_MILLIS(last_modified_time) AS last_modified_time,
dataset_id,
project_id
FROM `ironhacks-data.ironhacks_competition.__TABLES__`"""
query_job4 = bigquery_client.query(query4)
timestamp_data = query_job4.to_dataframe()
for cnt,row in timestamp_data.iterrows():
    print("\n")
    print(row["table_id"])
    print(row["creation_time"])
    print(row["last_modified_time"])

In [ ]:
##Printing (unique_vals, col name, datatype)
print(f'\nunemployment_data : No. of columns = {len(unemployment_data.columns)},Max week number: {max(unemployment_data["week_number"])},Min week number: {min(unemployment_data["week_number"])}')
print_column_info(unemployment_data)
print(f'\nwage_data : No. of columns = {len(wage_data.columns)}')
print_column_info(wage_data)
print(f'\nprediction_data : No. of columns = {len(prediction_data.columns)},Max week number: {max(prediction_data["week_number"])},Min week number: {min(prediction_data["week_number"])}')
print_column_info(prediction_data)

In [ ]:
count = 0
for uuid in unemployment_data["uu_id"].unique():
    if len(wage_data.loc[wage_data["uu_id"]==uuid])==0:
        count+=1
        #print(f'UUID {uuid} found in unemployment_data but not in wage_data')
print(f'{count} UUIDs found in unemployment_data but not in wage_data')
count = 0
for uuid in wage_data["uu_id"].unique():
    if len(unemployment_data.loc[unemployment_data["uu_id"]==uuid])==0:
        count+=1
        #print(f'UUID {uuid} found in unemployment_data but not in wage_data')
print(f'{count} UUIDs found in wage_data but not in unemployment_data')

In [ ]:
count = 0
for uuid in unemployment_data["uu_id"].unique():
    if len(merged_data.loc[merged_data["uu_id"]==uuid])==0:
        count+=1
        #print(f'UUID {uuid} found in unemployment_data but not in wage_data')
print(f'{count} UUIDs found in unemployment_data but not in merged_data')

In [ ]:
count = 0
for uuid in merged_data["uu_id"].unique():
    if len(unemployment_data.loc[unemployment_data["uu_id"]==uuid])==0:
        count+=1
        #print(f'UUID {uuid} found in unemployment_data but not in wage_data')
print(f'{count} UUIDs found in merged_data but not in unemployment_data')

In [ ]:
##Merging average wage with unemployment data
print(f'unemployment_data: Unique uu_id = {len(unemployment_data["uu_id"].unique())}, Row count = {len(unemployment_data.index)}, Col count = {len(unemployment_data.columns)}')
print(f'wage_data: Unique uu_id = {len(wage_data["uu_id"].unique())}, Row count = {len(wage_data.index)}, Col count = {len(wage_data.columns)}')
#print(f'Unique uu_id in wage_data = {len(wage_data["uu_id"].unique())}')
print(f'\nunemployment_data: NaN info, Total rows={len(unemployment_data.index)}')
print_na_info(unemployment_data)
print(f'merged_data: Unique uu_id = {len(merged_data["uu_id"].unique())}, Row count = {len(merged_data.index)}, Col count = {len(merged_data.columns)}')

In [ ]:
print(f'\nmerged_data : No. of columns = {len(merged_data.columns)},Max week number: {max(merged_data["week_number"])},Min week number: {min(merged_data["week_number"])}')
print_column_info(merged_data)
print(f'\nmerged_data: NaN info, Total rows={len(merged_data.index)}')
print_na_info(merged_data)

In [ ]:
#print(merged_data["week_number"])
merged_data["datetime"] = [get_datetime(val) for val in merged_data["week_number"]]
merged_data = merged_data.set_index(['datetime'])
merged_data = merged_data.sort_index()
merged_data.head(5)

In [ ]:
import csv
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import datetime,itertools
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

- DEFINE YOUR CLASSES AND FUNCTIONS 
-----------------------------------
This is not required, but is helpful in keeping your notebook organized. 
You can use the following cell or several cells to define your functions
and classes to keep them separate from your analysis or results code.
In general it useful to define your methods in a separate cell from where
it is run.

In [ ]:
def example_function():
    print('Hello World')

In [ ]:
def print_column_info(df):
    print(f'No. of columns: {len(df.columns)}')
    for col in df.columns:
        print(len(df[col].unique()),col,df[col].dtypes)
    print()

In [ ]:
def print_na_info(df):
    for col in df.columns:
        print(df[col].isnull().sum(),col,df[col].dtypes)

In [ ]:
def get_datetime(week_no):
    date = datetime.datetime.strptime("2022-"+str(week_no)+"-1","%Y-%W-%w")
    #print(date)
    return pd.to_datetime(date,format="%Y-%m-%d")

In [ ]:
def sarimax_gridsearch(ts, pdq, pdqs, maxiter=100, freq='D',disp=False):
    # Run a grid search with pdq and seasonal pdq parameters and get the best BIC value
    ans = []
    for comb in pdq:
        for combs in pdqs:
            #try:
            mod = SARIMAX(ts,order=comb,
                          seasonal_order=combs,
                          enforce_stationarity=False,
                          enforce_invertibility=False)

            output = mod.fit(maxiter=maxiter,disp=False) 
            ans.append([comb, combs, output.bic])
            #print('SARIMAX {} x {}12 : BIC Calculated ={}'.format(comb, combs, output.bic))
            #except:
            #    continue
    ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'bic'])
    ans_df = ans_df.sort_values(by=['bic'],ascending=True)[0:5]
    
    return ans_df

In [ ]:
def check_stationary(data,Print=0):
    adft = adfuller(data,autolag="AIC")
    output_df = pd.DataFrame({"Values":[adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]  , "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                                            "critical value (1%)", "critical value (5%)", "critical value (10%)"]})

    critical_value = adft[4]['5%']
    if Print==1:
        print(output_df)
    if adft[1] < 0.05 and adft[0] < critical_value:
        return 1
    else:
        return 0

In [ ]:
def diff_inv(series, last_observation):

    series_undifferenced = series.copy()

    series_undifferenced.iat[0] = series_undifferenced.iat[0] + last_observation

    series_undifferenced = series_undifferenced.cumsum()

    return series_undifferenced

In [ ]:
def loss(pred,actual):
    pred = np.round(pred)
    errors = abs(actual-pred)
    print(f'Mean Absolute Error: {round(np.mean(errors), 2)}')
    print(f'Mean squared Error: {round(np.mean(errors**2), 2)}')
    mape = 100 * (errors/actual)
    # Calcualte and display accuracy
    accuracy = 100 - np.mean(mape)
    print(f'Accuracy: {round(accuracy, 2)}%.')

In [ ]:
#print(merged_data["week_number"])
merged_data["datetime"] = [get_datetime(val) for val in merged_data["week_number"]]
merged_data = merged_data.set_index(['datetime'])
merged_data = merged_data.sort_index()
merged_data.head(5)

In [ ]:
print(f'merged_data: Unique uu_id = {len(merged_data["uu_id"].unique())}, Row count = {len(merged_data.index)}, Col count = {len(merged_data.columns)}')

In [ ]:
print(f'\nmerged_data : No. of columns = {len(merged_data.columns)},Max week number: {max(merged_data["week_number"])},Min week number: {min(merged_data["week_number"])}')
print_column_info(merged_data)
print(f'\nmerged_data: NaN info, Total rows={len(merged_data.index)}')
print_na_info(merged_data)

In [ ]:
entry_count = []
for uuid in merged_data["uu_id"].unique():
    entry_count.append([uuid,len(merged_data.loc[merged_data["uu_id"]==uuid])])
entry_count.sort(key=lambda x: x[1])
for entry in entry_count:
    print(f'uuid: {entry[0]}, weeks of data available: {entry[1]}')

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    #print(data_train)
    #SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(method="ffill"),
    #                       start_p=0, start_q=0,test="adf",
    #                       max_p=5, max_q=5,m=4,d=None,
    #                       trace=False,
    #                       suppress_warnings=True, 
    #                       maxiter=200,
    #                       stepwise=True,seasonal=True,D=None)
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,d=None,max_order=0)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
def sarimax_forecast(SARIMAX_model, periods=10,plot=0):
    # Forecast
    n_periods = periods

    forecast_df = data[data.index>=data_train.index[-1]]
    fitted, confint = SARIMAX_model.predict(n_periods=n_periods, 
                                            return_conf_int=True)
    #print(confint)
    index_of_fc = pd.date_range(data_train.index[-1], periods = n_periods, freq='W-MON')

    # make series for plotting purpose
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)
    
    # Plot
    if plot:
        plt.figure(figsize=(15,7))
        plt.plot(data["total_claims"], color='#1f76b4')
        plt.plot(fitted_series, color='darkgreen')
        plt.fill_between(lower_series.index, 
                        lower_series, 
                        upper_series, 
                        color='k', alpha=.15)

        plt.title("SARIMAX")
        plt.show()
    return fitted_series

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,d=None,max_order=0)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
pred_df = pd.DataFrame()
uuid_list = []
week_number_list = []
total_claims_list = []
#stat=[]
for l in output:
    uuid_list.append(l[0])
    week_number_list.append(l[1])
    if l[2]<0:
        l[2] = 0
    total_claims_list.append(int(np.floor(l[2])))
    #stat.append(l[1])
pred_df["uu_id"] = uuid_list
pred_df["week_number"] = week_number_list
pred_df["total_claims"] = total_claims_list
#pred_df["stationarity"] = stat

In [ ]:
pred = []
actual = []
pred_stat = []
actual_stat = []
pred_nonstat = []
actual_nonstat = []
ts_data = merged_data
for id,row in pred_df.iterrows():
    uuid = row["uu_id"]
    week_number = row["week_number"]
    if len(ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"])==0:
        continue
    actual_val = ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"].item()
    pred.append(row["total_claims"])
    actual.append(actual_val)

In [ ]:
loss(pred=pred,actual=actual)        

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,max_order=0)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
pred_df = pd.DataFrame()
uuid_list = []
week_number_list = []
total_claims_list = []
#stat=[]
for l in output:
    uuid_list.append(l[0])
    week_number_list.append(l[1])
    if l[2]<0:
        l[2] = 0
    total_claims_list.append(int(np.floor(l[2])))
    #stat.append(l[1])
pred_df["uu_id"] = uuid_list
pred_df["week_number"] = week_number_list
pred_df["total_claims"] = total_claims_list
#pred_df["stationarity"] = stat

In [ ]:
pred = []
actual = []
pred_stat = []
actual_stat = []
pred_nonstat = []
actual_nonstat = []
ts_data = merged_data
for id,row in pred_df.iterrows():
    uuid = row["uu_id"]
    week_number = row["week_number"]
    if len(ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"])==0:
        continue
    actual_val = ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"].item()
    pred.append(row["total_claims"])
    actual.append(actual_val)

In [ ]:
loss(pred=pred,actual=actual)        

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,d=None)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
pred_df = pd.DataFrame()
uuid_list = []
week_number_list = []
total_claims_list = []
#stat=[]
for l in output:
    uuid_list.append(l[0])
    week_number_list.append(l[1])
    if l[2]<0:
        l[2] = 0
    total_claims_list.append(int(np.floor(l[2])))
    #stat.append(l[1])
pred_df["uu_id"] = uuid_list
pred_df["week_number"] = week_number_list
pred_df["total_claims"] = total_claims_list
#pred_df["stationarity"] = stat

In [ ]:
pred = []
actual = []
pred_stat = []
actual_stat = []
pred_nonstat = []
actual_nonstat = []
ts_data = merged_data
for id,row in pred_df.iterrows():
    uuid = row["uu_id"]
    week_number = row["week_number"]
    if len(ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"])==0:
        continue
    actual_val = ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"].item()
    pred.append(row["total_claims"])
    actual.append(actual_val)

In [ ]:
loss(pred=pred,actual=actual)        

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,d=None,maxiter=200)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
ids = merged_data["uu_id"].unique()[:]
data = merged_data.loc[merged_data["uu_id"]==ids[500]].copy()
print(data["week_number"].max())
data_train = data[data["week_number"]<32]
data_train = data_train.asfreq('W-MON')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"])
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0))
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=1)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
plt.plot(trend)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=1)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=3)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0))
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=20)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=10)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=10)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=15)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=12)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=15)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(data_train["total_claims"].fillna(0))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
ax = seasonal.plot(label='Seasonality', color='blue')
min_ = seasonal.idxmin()
max_ = seasonal.idxmax()
min_2 = seasonal[max_:].idxmin()
max_2 = seasonal[min_2:].idxmax()
ax.axvline(min_,label='min 1',c='red')
ax.axvline(min_2,label='min 2',c='red', ls=':')
ax.axvline(max_,label='max 1',c='green')
ax.axvline(max_2,label='max 2',c='green', ls=':')
plt.legend(loc='upper right', fontsize='x-small')
print(f'The time difference between the two minimums is {min_2-min_}')

In [ ]:
def sarimax_forecast(SARIMAX_model, periods=10,plot=0):
    # Forecast
    n_periods = periods

    forecast_df = data[data.index>=data_train.index[-1]]
    fitted, confint = SARIMAX_model.predict(n_periods=n_periods, 
                                            return_conf_int=True)
    #print(confint)
    index_of_fc = pd.date_range(data_train.index[-1], periods = n_periods, freq='W-MON')

    # make series for plotting purpose
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)
    
    # Plot
    if plot:
        plt.figure(figsize=(15,7))
        plt.plot(data["total_claims"], color='#1f76b4')
        plt.plot(fitted_series, color='darkgreen')
        plt.fill_between(lower_series.index, 
                        lower_series, 
                        upper_series, 
                        color='k', alpha=.15)

        plt.title("SARIMAX")
        plt.show()
    return fitted_series

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,m=14,maxiter=200)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),alpha=0.05,m=14)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5, m=12, max_order=None,
                           trace=True)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5, m=12, max_order=None)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5, m=None, max_order=None)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5,  max_order=None)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
pred_df = pd.DataFrame()
uuid_list = []
week_number_list = []
total_claims_list = []
#stat=[]
for l in output:
    uuid_list.append(l[0])
    week_number_list.append(l[1])
    if l[2]<0:
        l[2] = 0
    total_claims_list.append(int(np.floor(l[2])))
    #stat.append(l[1])
pred_df["uu_id"] = uuid_list
pred_df["week_number"] = week_number_list
pred_df["total_claims"] = total_claims_list
#pred_df["stationarity"] = stat

In [ ]:
pred = []
actual = []
pred_stat = []
actual_stat = []
pred_nonstat = []
actual_nonstat = []
ts_data = merged_data
for id,row in pred_df.iterrows():
    uuid = row["uu_id"]
    week_number = row["week_number"]
    if len(ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"])==0:
        continue
    actual_val = ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"].item()
    pred.append(row["total_claims"])
    actual.append(actual_val)

In [ ]:
loss(pred=pred,actual=actual)        

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5, m=7, max_order=None)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
exog = ['edu_8th_or_less', 'edu_grades_9_11',
       'edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown',
       'top_category_employer1', 'top_category_employer2',
       'top_category_employer3', 'gender_female', 'gender_male', 'gender_na',
       'race_amerindian', 'race_asian', 'race_black', 'race_noanswer',
       'race_hawaiiannative', 'race_other', 'race_white', 'average_wage']
exog = ['edu_8th_or_less', 'edu_grades_9_11','edu_hs_grad_equiv', 'edu_post_hs', 'edu_unknown']
ids = merged_data["uu_id"].unique()[:]
output = []
for cnt,idd in enumerate(ids):
    if cnt > 100: break
    data = merged_data.loc[merged_data["uu_id"]==idd].copy()
    data = data.asfreq('W-MON')
    is_na = []
    for idx,row in data.iterrows():
        if pd.isna(row["total_claims"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data["is_na"] = is_na
    #data["total_claims"] = data["total_claims"].fillna(0)
    start_week = 0
    #print(data["week_number"].max())
    boundary_week = data["week_number"].max()-2
    data_train = data.loc[(data["week_number"]<boundary_week) & (data["week_number"]>start_week)].drop("is_na",axis=1)
    is_na = []
    for idx,row in data_train.iterrows():
        if pd.isna(row["week_number"]):
            #print(f'NA found in week {idx}')
            is_na.append(1)
        else:
            is_na.append(0)
    data_train["is_na"] = is_na
    data_train = data_train.asfreq('W-MON')
    SARIMAX_model = pm.auto_arima(data_train["total_claims"].fillna(0),start_p=0, start_q=0, max_p=5,
                           max_q=5, start_P=0, start_Q=0, max_P=5,
                           max_Q=5, max_order=None,seasonal=False)
    max_week_no = data["week_number"].max()
    pred_periods =  max_week_no - data_train["week_number"].max()
    #SARIMAX_model_fit = SARIMAX_model.fit(data_train["total_claims"].fillna(0))
    #check_st = check_stationary(data_train["total_claims"].fillna(0),1)
    prediction = sarimax_forecast(SARIMAX_model, periods=int(pred_periods)+1,plot=0)
    out = prediction.loc[prediction.index[-1]]
    #print(prediction)
    #print(prediction.loc[prediction.index[-1]])
    output.append([idd,max_week_no,out])
    #print(check_stationary(data_train["total_claims"].fillna(method="bfill"),1))
    if len(data.loc[data["week_number"]==max_week_no,"total_claims"])>0:
        print(idd,max_week_no,out,data.loc[data["week_number"]==max_week_no,"total_claims"].item(),"--",len(data_train.index))
    else:
        print(idd,max_week_no,out,"--",len(data_train.index))
    #break

In [ ]:
pred_df = pd.DataFrame()
uuid_list = []
week_number_list = []
total_claims_list = []
#stat=[]
for l in output:
    uuid_list.append(l[0])
    week_number_list.append(l[1])
    if l[2]<0:
        l[2] = 0
    total_claims_list.append(int(np.floor(l[2])))
    #stat.append(l[1])
pred_df["uu_id"] = uuid_list
pred_df["week_number"] = week_number_list
pred_df["total_claims"] = total_claims_list
#pred_df["stationarity"] = stat

In [ ]:
pred = []
actual = []
pred_stat = []
actual_stat = []
pred_nonstat = []
actual_nonstat = []
ts_data = merged_data
for id,row in pred_df.iterrows():
    uuid = row["uu_id"]
    week_number = row["week_number"]
    if len(ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"])==0:
        continue
    actual_val = ts_data.loc[(ts_data["uu_id"]==uuid) & (ts_data["week_number"] == week_number),"total_claims"].item()
    pred.append(row["total_claims"])
    actual.append(actual_val)

In [ ]:
loss(pred=pred,actual=actual)        

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0).diff(),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].diff(),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].diff().fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].diff().diff().fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].diff().diff().diff().fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
ids = merged_data["uu_id"].unique()[:]
data = merged_data.loc[merged_data["uu_id"]==ids[500]].copy()
print(data["week_number"].max())
data_train = data[data["week_number"]<32]
#data_train = data_train.asfreq('W-MON')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"],period=14)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"])
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
ids = merged_data["uu_id"].unique()[:]
data = merged_data.loc[merged_data["uu_id"]==ids[500]].copy()
print(data["week_number"].max())
data_train = data[data["week_number"]<32]
#data_train = data_train.asfreq('W-MON')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"])
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"],period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
ids = merged_data["uu_id"].unique()[:]
data = merged_data.loc[merged_data["uu_id"]==ids[500]].copy()
print(data["week_number"].max())
data_train = data[data["week_number"]<32]
data_train = data_train.asfreq('W-MON')

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
# Taking the decomposition
decomposition = seasonal_decompose(data_train["total_claims"].fillna(0),period=7)
# Gathering and plotting the trend, seasonality, and residuals 
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
decomposition.plot()
print(len(data_train["total_claims"].fillna(0)))

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = [i for i in range(len(data_train["total_claims"].fillna(0)))]
y = data_train["total_claims"].fillna(0)
model.fit(X,y)

In [ ]:
trend = model.predict(X)
plt.plot(y)
plt.plot(trend)
plt.plot(show)

detrended = 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = [i for i in range(0,len(data_train["total_claims"].fillna(0)))]
y = data_train["total_claims"].fillna(0)
model.fit(X,y)

In [ ]:
trend = model.predict(X)
plt.plot(y)
plt.plot(trend)
plt.plot(show)

detrended = 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = [i for i in range(0,len(data_train["total_claims"].fillna(0)))].reshape(-1,1)
y = data_train["total_claims"].fillna(0)
model.fit(X,y)

In [ ]:
trend = model.predict(X)
plt.plot(y)
plt.plot(trend)
plt.plot(show)

detrended = 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = [i for i in range(0,len(data_train["total_claims"].fillna(0)))]
X = np.reshape(X,(len(X),1))
y = data_train["total_claims"].fillna(0)
model.fit(X,y)

In [ ]:
trend = model.predict(X)
plt.plot(y)
plt.plot(trend)
plt.plot(show)

detrended = 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()
X = [i for i in range(0,len(data_train["total_claims"].fillna(0)))]
X = np.reshape(X,(len(X),1))
y = data_train["total_claims"].fillna(0)
model.fit(X,y)

In [ ]:
trend = model.predict(X)
plt.plot(y)
plt.plot(trend)
plt.show()

detrended = 